In [2]:
!pip3.10 install --upgrade guidance==0.0.59 -qq
import os, requests

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" # can be anything
# os.environ["OPENAI_API_BASE"] = "http://192.168.1.76:8081/v1"
# os.environ["OPENAI_API_HOST"] = "http://192.168.1.76:8081"

import getpass, os 
secret_key = getpass.getpass('Enter OpenAI secret key: ') 
os.environ['OPENAI_API_KEY'] = secret_key


import guidance

# set the default language model used to execute guidance programs
guidance.llm = guidance.llms.OpenAI("text-davinci-003")

!pip show guidance

DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/python_docx-0.8.11-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/pydantic-1.10.7-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/typing_extensions-4.5.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/pytest-7.2.2-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/typing_inspect-0.8.0-py3.11.egg i

To use the official OpenAI server

In [3]:
# import getpass, os 
# secret_key = getpass.getpass('Enter OpenAI secret key: ') 
# os.environ['OPENAI_API_KEY'] = secret_key

## Define DB query functions

In [18]:
def get_similar_sentences_to_verse(query_string, target_language_code, limit):
    """Query local endpoint http://localhost:3000/api/query/{target_language_code}/{query_string}&limit={number_of_results}"""
    url = f'http://localhost:3000/api/query/{target_language_code}/{query_string}&limit={limit}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {'error': response.status_code}
    
def get_translation_prompt_data(vref, target_language_code, limit=5):
    '''http://localhost:3000/api/translation-prompt-builder?vref={url_encoded_vref = e.g., ROM%201%3A1}&target_language_code={target_language_code}'''
    url = f'http://localhost:3000/api/translation-prompt-builder?vref={vref}&target_language_code={target_language_code}&number_of_examples={limit}'
    response = requests.get(url)
    if response.status_code == 200:
        values = []
        """Each item looks like this:
        'JAS 1:1': {'source': 'Ἰάκωβος  Θεοῦ  καὶ  Κυρίου  Ἰησοῦ  Χριστοῦ  δοῦλος  ταῖς  δώδεκα  φυλαῖς  ταῖς  ἐν  τῇ  Διασπορᾷ  χαίρειν.',
  'bridge_translation': 'James, a servant of God and of the Lord Jesus Christ, To the twelve tribes of the Dispersion: Greetings.',
  'target': 'Baitumatumayah etei a tafaram kwaihamiy kwatit tafaram tata’amaim kwama’am, etei a merar ayiy. Ayu James God ana akir wairafin na’atube ata Regah Jesu Keriso ana akir wairafin kwa a fef akikirum.'},
        
        I want the key to be a 'vref' property inside the object
        """
        for key, value in response.json().items():
            values.append({**value, 'vref': f'{key}'})
        return values
    else:
        return [{'error': response.status_code}]

# get unique tokens from /api/unique_tokens?language_code={target_language_code}
def get_unique_tokens(target_language_code):
    """Query local endpoint http://localhost:3000/api/unique_tokens?language_code={target_language_code}"""
    url = f'http://localhost:3000/api/unique_tokens?language_code={target_language_code}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {'error': response.status_code}
    
def get_ngrams(target_language_code, size, n):
    """Query local endpoint http://localhost:3000/api/ngrams?language_code={target_language_code}"""
    url = f'http://localhost:3000/api/ngrams?language_code={target_language_code}&size={size}&n={n}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {'error': response.status_code}
    
# Get the plain treedown representation for a token's sentence

# example endpoint: "https://labs.clear.bible/symphony-dev/api/GNT/Nestle1904/lowfat?usfm-ref=JHN%2014:1" - JHN 14:1

from lxml import etree

def process_element(element, usfm_ref, indent=0):
    treedown_str = ""
    
    if element.get("class") == "cl":
        treedown_str += "\n" + "  " * indent

    if element.get("role"):
        role = element.attrib["role"]
        if role == "adv":
            role = '+'
        treedown_str += "\n" + "  " * indent + role + ": "
        
    # bold the matching token using usfm ref
    if element.tag == "w" and element.get("ref") == usfm_ref:
        treedown_str += "**" + element.text + "**"
        treedown_str += element.attrib.get("after", "") + ' '

    if element.tag == "w" and element.text:
        treedown_str += element.attrib.get("gloss", "") + f'[{element.text}]'
        treedown_str += element.attrib.get("after", "") + ' '

    for child in element:
        treedown_str += process_element(child, usfm_ref, indent + 1)

    return treedown_str

def process_element_clauses(element, indent=0):
    greek_text = ""
    gloss_text = ""
    
    if element.get("class") == "cl":
        greek_text += "\n" + ("  " * indent)
        gloss_text += "\n" + ("  " * indent)
        
    if element.tag == "w" and element.text:
        greek_text += element.text + " "
        gloss_text += element.attrib.get("gloss", "") + " "

    for child in element:
        # # Check if the child is a nested clause
        # if child.get("class") == "clause":
        #     # If it is, we want to end the current clause, and process the child with indent + 1
        #     child_greek_text, child_gloss_text = process_element_clauses(child, indent + 1)
        #     greek_text += child_greek_text
        #     gloss_text += child_gloss_text
        # else:
            # If it's not, we want to process the child with the same indent
        child_greek_text, child_gloss_text = process_element_clauses(child, indent + 1)
        greek_text += child_greek_text
        gloss_text += child_gloss_text

    return greek_text, gloss_text

def processing_router(element, usfm_ref, indent=0, clauses_only=False):
    if clauses_only:
        return process_element_clauses(element, indent)
    else:
        return process_element(element, usfm_ref, indent)

def get_treedown_by_ref(usfm_ref):
    print('Getting treedown for ref', usfm_ref)
    usfm_passage = usfm_ref.split('!')[0]
    endpoint = "https://labs.clear.bible/symphony-dev/api/GNT/Nestle1904/lowfat?usfm-ref=" + usfm_passage
    
    # Note: the response is XML like this:
    '''
    <sentences xml:lang="grc" ref="JHN 14:1">
        <sentence>
        <p>
        <milestone unit="verse" id="JHN 14:1">JHN 14:1</milestone>
        Μὴ ταρασσέσθω ὑμῶν ἡ καρδία·
        </p>
        <wg>
        <wg class="cl" rule="ADV-V-S">
        <w role="adv" ref="JHN 14:1!1" after=" " class="adv" id="n43014001001" lemma="μή" normalized="Μή" strong="3361" gloss="Not" domain="069002" ln="69.3" morph="PRT-N" unicode="Μὴ">Μὴ</w>
        ...
    '''
    text_response = requests.get(endpoint).text
    # print(text_response)
    
    xml = etree.fromstring(text_response.encode('utf-8'))
    # turn xml into simple treedown, with all text on one line, except a new line for <wg class="cl".../> elements, and a new indented line for <w role.../> elements
    
    # treedown = processing_router(xml, usfm_ref, indent=0, clauses_only=False)
    greek_text, gloss_text = processing_router(xml, usfm_ref, indent=0, clauses_only=True)
    treedown = f'{greek_text}\n{gloss_text}\n\n'
    return treedown
# test

print(get_treedown_by_ref('MAT 3:15'))

Getting treedown for ref MAT 3:15
δὲ 
        
          ἀποκριθεὶς ὁ Ἰησοῦς εἶπεν αὐτῷ 
          Ἄφες ἄρτι γὰρ 
              οὕτως 
                πρέπον ἐστὶν ἡμῖν 
                πληρῶσαι πᾶσαν δικαιοσύνην 
      τότε ἀφίησιν αὐτόν 
however 
        
          Answering - Jesus said him 
          Permit [it] presently for 
              thus 
                fitting it is to us 
                to fulfill all righteousness 
      Then he permits Him 




In [4]:
# get_unique_tokens(target_language_code)

In [5]:
# # get_similar_sentences_to_verse('hi, this is a test sentence.', 'bsb_bible', 5)
# get_translation_prompt_data('ROM 1:1', 'aai')

In [19]:
import json
with open('/Users/ryderwishart/translators-copilot/data/bible/fraLSG.json', 'r') as f:
    triplets = json.load(f)

print(len(triplets))

31102


In [20]:
first_nt_verse_index = 23145

In [21]:
def treedown_lines(index):
    import re
    lines = get_treedown_by_ref(triplets[index]['vref']).split('\n')
    # skip empty lines and lines that are just `[a-z+]: `, e.g. `+:` or `n: ` (use regex)
    return [line.strip().removeprefix(' ') for line in lines if line.strip() != '' and not re.match(r'^[a-z+]:$', line.strip())]

treedown_lines(first_nt_verse_index + 100)

Getting treedown for ref MAT 5:11


['μακάριοί ἐστε ὅταν',
 'ὀνειδίσωσιν ὑμᾶς καὶ',
 'διώξωσιν καὶ',
 'εἴπωσιν πᾶν πονηρὸν καθ’ ὑμῶν',
 'ψευδόμενοι ἕνεκεν ἐμοῦ',
 'Blessed are you when',
 'they shall insult you and',
 'shall persecute [you] and',
 'shall say all kinds of evil against you',
 'lying on account of Me']

## Load Guidance and sanity check

In [9]:
# import guidance

# llm = guidance.llms.OpenAI(
#         'text-davinci-003', # This is a required param, and Guidance validates the name
#         # endpoint = "http://192.168.1.76:8081/v1/"
#     )

# def reset_llm():
#     llm.cache.clear()

# # Sanity check:
# prog = guidance('hello {{name}} {{gen "something" stop=";"}}', llm=llm)(name='world!')


## Using line-by-line breakdown with token-output restriction

In [41]:
# Let's use treedown_lines(first_nt_verse_index + 100) as the text (which returns an array of lines)
# and then let's use the following program to generate an alignment for every single line

alignment_program = guidance('''Source/English: {{content}}
# French: {{select 'alignment' options=target_options}}''')

treedown_alignment_program = guidance('''USER: Please also align the following sentence phrase by phrase:

English: {{triplet['bsb']['content']}}
Source: {{triplet['macula']['content']}}
French: {{triplet['target']['content']}}

ASSISTANT: Here is a phonological, semantic, orthographic alignment of the tokens in the sentence:
{{#each lines}}
{{>alignment_program}}
{{/each}}
''')

current_verse_index = first_nt_verse_index + 101

lines = treedown_lines(current_verse_index)
lines = [{'content': line} for line in lines]

triplet = triplets[current_verse_index]
import re
from nltk.util import ngrams

target_tokens = re.split(' ', triplet['target']['content'])
target_tokens = [token for token in target_tokens if token != '']

target_options = target_tokens[:]
for n in range(2, min(6, len(target_tokens) + 1)):
    target_options.extend([' '.join(ngram) for ngram in ngrams(target_tokens, n)])
assert len(lines) > 0

for line in lines:
    output = alignment_program(content=line['content'], target_options=target_options)
    selected_option = output['alignment']
    target_options = [option for option in target_options if selected_option not in option]

output = treedown_alignment_program(triplet=triplet, lines=lines)


Stop program Source/English: χαίρετε καὶ 
# French:

Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/guidance/_program_executor.py", line 102, in run
    await self.visit(self.parse_tree)
  File "/opt/homebrew/lib/python3.10/site-packages/guidance/_program_executor.py", line 437, in visit
    visited_children.append(await self.visit(child, inner_next_node, inner_next_next_node, inner_prev_node, node, parent_node))
  File "/opt/homebrew/lib/python3.10/site-packages/guidance/_program_executor.py", line 437, in visit
    visited_children.append(await self.visit(child, inner_next_node, inner_next_next_node, inner_prev_node, node, parent_node))
  File "/opt/homebrew/lib/python3.10/site-packages/guidance/_program_executor.py", line 226, in visit
    visited_children = [await self.visit(child, next_node, next_next_node, prev_node, node, parent_node) for child in node.children]
  File "/opt/homebrew/lib/python3.10/site-packages/guidance/_program_executor.py", line 226, in <listcomp>
    visited_children = [a

AssertionError: No logprobs returned from LLM!, {
  "text_offset": [],
  "token_logprobs": [],
  "tokens": [],
  "top_logprobs": []
}

In [ ]:
output._exception

In [50]:
import openai
import requests

import tiktoken

enc = tiktoken.encoding_for_model("gpt-4")

# Tokenize the French sentence
def tokenize(sentence):
    return enc.encode(sentence)

# Get the token IDs
french_sentence = triplet['target']['content']
tokenized_french = tokenize(french_sentence)
print(tokenized_french)
token_ids = tokenized_french

# Create a bias map with the token IDs
bias_map = {str(token_id): 100 for token_id in token_ids}


current_verse_index = first_nt_verse_index + 100

lines = treedown_lines(current_verse_index)
lines = [{'content': line} for line in lines]

triplet = triplets[current_verse_index]

# Generate the completion
source_structure = f'''USER: Please also align the following sentence phrase by phrase:

English: {triplet['bsb']['content']}
Source: {triplet['macula']['content']}
French: {triplet['target']['content']}

Source clauses:
{lines[0]['content']}'''

response = openai.Completion.create(
  engine="gpt-3.5-turbo-instruct",
  prompt=source_structure,
  max_tokens=60,
  temperature=0.5,
  logit_bias=bias_map
)

print(response.choices[0].text.strip())

[85936, 73, 283, 15760, 89, 45325, 1880, 779, 9188, 89, 7010, 326, 92748, 978, 18297, 325, 11, 66021, 1744, 15265, 9517, 5807, 1137, 35737, 25626, 7010, 3625, 272, 53819, 26, 1841, 272, 22117, 44351, 934, 68306, 264, 7565, 978, 10453, 978, 3625, 40334, 4558, 2392, 7930, 14848, 24560, 33670, 9189, 13]
Getting treedown for ref MAT 5:11
, réjouissez-vous, vous, et, qu’on vous, qu’on vous, et, qu’on, qu’on, et, qu’on, qu’on, et, qu’on, qu’on, et, qu’on, qu’on, et, qu’on, qu’on,
